In [ ]:
# Hugging Face Transformer fine tuning tutorial
# https://huggingface.co/docs/transformers/training

# two parts 1. using the trainer class  - the nitty gritty of how this works is
# hidden in the trainer class.
#  The tutorial as given on the hugginface site back propagates through all
# layers of the network, which is slow and, by my experience and understanding
# of the literature, will produce suboptimal results.
# The scripts below have added some additional code to explore the model
# and to freeze the lower layers.
# I've also reduce the size of the training and test sets slightly to speed up
# experiments with the code.
# This all seems to work, but still playing with it to make things clearer.

#           2. Using native pyTorch to write the training loop etc. - probably more
#              informative about what is going on. Not started this yet.

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install datasets

In [ ]:
import torch


In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
len(tokenized_datasets["train"])

650000

In [ ]:
len(tokenized_datasets["test"])

50000

In [ ]:
# reduce the training time by using a smaller training and test set.
# The original tutorial selects only 1000 but I've reduced this further to 800
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(800))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(800))

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install accelerate -U

In [ ]:
# save current model.
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MyDrive


In [ ]:
# 1. Create models directory
MODEL_PATH = Path("/content/drive/MyDrive/Colab Notebooks/hugging_face/test_trainer")
MODEL_PATH.mkdir(parents=True, exist_ok=True)



In [ ]:
from transformers.training_args import TrainingArguments

training_args = TrainingArguments(output_dir=MODEL_PATH)

In [ ]:
!pip install evaluate

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# let's explore the model - this is not in the original tutorial.
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# # This is not in the hugging face tutorial :- summarise the model to see what to retrain
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

In [ ]:
summary(model=model, col_names=[ "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                                Param #              Trainable
BertForSequenceClassification (BertForSequenceClassification)          --                   True
├─BertModel (bert)                                                     --                   True
│    └─BertEmbeddings (embeddings)                                     --                   True
│    │    └─Embedding (word_embeddings)                                22,268,928           True
│    │    └─Embedding (position_embeddings)                            393,216              True
│    │    └─Embedding (token_type_embeddings)                          1,536                True
│    │    └─LayerNorm (LayerNorm)                                      1,536                True
│    │    └─Dropout (dropout)                                          --                   --
│    └─BertEncoder (encoder)                                           --                   True
│    │    └─ModuleList (lay

In [ ]:
# This is not in the hugging face tutorial - not sure why they retrain the whole model?
num_layers_to_freeze = 2
for param in model.base_model.parameters():
    param.requires_grad = False

In [ ]:
summary(model=model, col_names=[ "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                                Param #              Trainable
BertForSequenceClassification (BertForSequenceClassification)          --                   Partial
├─BertModel (bert)                                                     --                   False
│    └─BertEmbeddings (embeddings)                                     --                   False
│    │    └─Embedding (word_embeddings)                                (22,268,928)         False
│    │    └─Embedding (position_embeddings)                            (393,216)            False
│    │    └─Embedding (token_type_embeddings)                          (1,536)              False
│    │    └─LayerNorm (LayerNorm)                                      (1,536)              False
│    │    └─Dropout (dropout)                                          --                   --
│    └─BertEncoder (encoder)                                           --                   False
│    │    └─Modul

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=300, training_loss=1.6144486490885417, metrics={'train_runtime': 7668.291, 'train_samples_per_second': 0.313, 'train_steps_per_second': 0.039, 'total_flos': 631483541913600.0, 'train_loss': 1.6144486490885417, 'epoch': 3.0})